## Example of generating necessary FCC crystal data from Onsager code to run the symmetric neural networks for 5x5x6 orthogonal supercells.

In [1]:
from onsager import crystal, supercell, cluster
import numpy as np
import h5py

In [2]:
a0 = 1.0
basis_cube_fcc = [np.array([0, 0, 0]), np.array([0, 0.5, 0.5]), np.array([0.5, 0., 0.5]), np.array([0.5, 0.5, 0.])]
crys = crystal.Crystal(lattice=np.eye(3)*a0, basis=[basis_cube_fcc], chemistry=["A"], noreduce=True)
print(crys)

#Lattice:
  a1 = [1. 0. 0.]
  a2 = [0. 1. 0.]
  a3 = [0. 0. 1.]
#Basis:
  (A) 0.0 = [0. 0. 0.]
  (A) 0.1 = [0.  0.5 0.5]
  (A) 0.2 = [0.5 0.  0.5]
  (A) 0.3 = [0.5 0.5 0. ]


In [3]:
a0 = 1.0
cut = 1.01*a0/np.sqrt(2)  # Nearest neighbor cutoff + tolerance of 1%
fcc_primitive = crystal.Crystal.FCC(a0, chemistry="A")
lattice_primitive = fcc_primitive.lattice
jnetFCC = fcc_primitive.jumpnetwork(0, cut)

N_units = 5
superlatt = np.eye(crys.dim, dtype=int)*N_units
superFCC = supercell.ClusterSupercell(crys, superlatt)
print(superFCC.lattice)
print(len(superFCC.mobilepos))
#we'll put the vacancy at cartesian position 0,0,0
x_site = np.zeros(3)
R, ci = crys.cart2pos(x_site)
vacsiteInd = superFCC.index(R, ci)[0]
vacsiteInd

[[5. 0. 0.]
 [0. 5. 0.]
 [0. 0. 5.]]
500


0

In [4]:
jnetFCC[0]

[((0, 0), array([ 0. , -0.5, -0.5])),
 ((0, 0), array([-0. ,  0.5,  0.5])),
 ((0, 0), array([ 0. , -0.5,  0.5])),
 ((0, 0), array([-0. ,  0.5, -0.5])),
 ((0, 0), array([-0.5, -0.5,  0. ])),
 ((0, 0), array([ 0.5,  0.5, -0. ])),
 ((0, 0), array([-0.5,  0. ,  0.5])),
 ((0, 0), array([ 0.5, -0. , -0.5])),
 ((0, 0), array([-0.5,  0. , -0.5])),
 ((0, 0), array([ 0.5, -0. ,  0.5])),
 ((0, 0), array([ 0.5, -0.5,  0. ])),
 ((0, 0), array([-0.5,  0.5, -0. ]))]

In [5]:
print(superFCC.crys.dim)

3


In [6]:
# Next, for each site, store the nearest neighbors
N_ngb = len(jnetFCC[0])
Nsites = len(superFCC.mobilepos)
print(Nsites)
NNList = np.zeros((N_ngb + 1, Nsites), dtype=int)

dxList_1nn = np.array([j[1] for j in jnetFCC[0]])
print(dxList_1nn)

for siteInd in range(Nsites):
    # First, store the site as its own 0th neighbor
    NNList[0, siteInd] = siteInd
    ciSite, Rsite = superFCC.ciR(siteInd)
    x_site = superFCC.crys.pos2cart(Rsite, ciSite)
    
    for jInd in range(dxList_1nn.shape[0]):
        x_NN = x_site + dxList_1nn[jInd]
        R_NN, ci_NN = superFCC.crys.cart2pos(x_NN)
        idx_NN = superFCC.index(R_NN, ci_NN)[0]
        
        NNList[jInd + 1, siteInd] = idx_NN

500
[[ 0.  -0.5 -0.5]
 [-0.   0.5  0.5]
 [ 0.  -0.5  0.5]
 [-0.   0.5 -0.5]
 [-0.5 -0.5  0. ]
 [ 0.5  0.5 -0. ]
 [-0.5  0.   0.5]
 [ 0.5 -0.  -0.5]
 [-0.5  0.  -0.5]
 [ 0.5 -0.   0.5]
 [ 0.5 -0.5  0. ]
 [-0.5  0.5 -0. ]]


In [7]:
## Since the onsager code does not store group operations in any particular order,
## We'll give them indices and store them for further use.
GIndtoGDict = {}
count = 0
for g in crys.G:
    # Check if the group op is identity - give it the zero index - not necessary, just a choice
    if np.allclose(g.cartrot, np.eye(3)):
        GIndtoGDict[0] = g
    else:
        count += 1
        GIndtoGDict[count] = g

In [8]:
len(GIndtoGDict)

48

In [9]:
# Next, store the cartesian rotation matrices of the group operations.
GroupOpLatticeCartRotMatrices = np.zeros((len(GIndtoGDict), 3, 3))
for key in GIndtoGDict.keys():
    GroupOpLatticeCartRotMatrices[key, :, :] = GIndtoGDict[key].cartrot

In [10]:
# Now for each group operation, store, the permutation of the nearest neighbors due to its inverse
GpermNNIdx = np.zeros((len(GIndtoGDict), dxList_1nn.shape[0] + 1), dtype=int)
for gInd, g in GIndtoGDict.items():
    for jInd in range(dxList_1nn.shape[0]):
        dx = dxList_1nn[jInd]
        dxRot = superFCC.crys.g_cart(g.inv(), dx)
        count = 0
        idxRot = None
        for jIndRot in range(dxList_1nn.shape[0]):
            if np.allclose(dxRot, dxList_1nn[jIndRot], atol=1e-8, rtol=0):
                idxRot = jIndRot
                count += 1
        
        assert count == 1
        GpermNNIdx[gInd, jInd + 1] = idxRot + 1

In [11]:
ciVac, RVac = superFCC.ciR(vacsiteInd)
print(ci, R)
x_vac = superFCC.crys.pos2cart(RVac, ciVac)
print(x_vac)

(0, 0) [0 0 0]
[0. 0. 0.]


In [12]:
# store how each jump changes occupancies, if we keep the vacancy at the (cartesian) origin in both states
jumpSiteIndex = np.zeros((dxList_1nn.shape[0], Nsites), dtype=int)
jumpSiteIndex[:, vacsiteInd] = vacsiteInd
print(x_vac, vacsiteInd)
for jumpInd in range(dxList_1nn.shape[0]):
    
    dxJump = dxList_1nn[jumpInd]
    
    x_site_exchange = x_vac + dxJump
    Rsite_exchange, ciSite_exchange = superFCC.crys.cart2pos(x_site_exchange)
    siteExchange = superFCC.index(Rsite_exchange, ciSite_exchange)[0]
    
    assert siteExchange == NNList[1+jumpInd, vacsiteInd]
    
    x_site_exchange_new = x_vac - dxJump # Relative to the vacancy, this site moves to the opposite site
    Rsite_exchange_new, ciSite_exchange_new = superFCC.crys.cart2pos(x_site_exchange_new)
    siteExchangeNew = superFCC.index(Rsite_exchange_new, ciSite_exchange_new)[0]
    
    jumpSiteIndex[jumpInd, siteExchangeNew] = siteExchange  # the exchanged site
    
    for siteInd in range(Nsites):
        # vacancy doesn't move so don't include it
        # Exchange site already done so don't include it either
        if siteInd == siteExchange or siteInd == vacsiteInd:
            continue
        
        ciSite, Rsite = superFCC.ciR(siteInd)
        xSite = superFCC.crys.pos2cart(Rsite, ciSite)
        
        # Translate the site by the negative of the jump vector
        xsiteNew = xSite - dxJump
        
        RsiteNew, ciSiteNew = superFCC.crys.cart2pos(xsiteNew)
        
        siteIndNew, _ = superFCC.index(RsiteNew, ciSiteNew)
        
        assert siteIndNew != vacsiteInd
        assert siteIndNew != siteExchangeNew
        
        jumpSiteIndex[jumpInd, siteIndNew] = siteInd

[0. 0. 0.] 0


In [13]:
superCellSiteToCart = np.zeros((Nsites, superFCC.crys.dim))
for siteInd in range(Nsites):
    ciSite, Rsite = superFCC.ciR(siteInd)
    xSite = superFCC.crys.pos2cart(Rsite, ciSite)
    
    assert np.allclose(xSite, np.dot(superFCC.lattice, superFCC.mobilepos[siteInd]))
    superCellSiteToCart[siteInd, :] = xSite[:]

In [14]:
with h5py.File("CrystData_ortho_{}_cube.h5".format(N_units), "w") as fl:
    fl.create_dataset("Lattice_basis_vectors", data=superFCC.crys.lattice)
    fl.create_dataset("SuperLatt", data=superlatt)
    fl.create_dataset("GroupOpLatticeCartRotMatrices", data=GroupOpLatticeCartRotMatrices)
    fl.create_dataset("dxList_1nn", data=dxList_1nn)
    fl.create_dataset("NNsiteList_sitewise", data=NNList)
    fl.create_dataset("JumpSiteIndexPermutation", data=jumpSiteIndex)
    fl.create_dataset("GroupNNPermutation", data=GpermNNIdx)
    
    # Other than this, let's also store the cartesian positions of the sites
    # and the basis of the cubic crystal
    fl.create_dataset("basis_sites", data=np.array(basis_cube_fcc))
    fl.create_dataset("superCellSiteToCart", data=superCellSiteToCart)